In [1]:
from termcolor import colored

import math as m

import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Rectangle

import ipywidgets as widgets

In [10]:
EXT_SIZE = 60
INT_SIZE = 20

EXT_BORDER = +300.0
INT_BORDER = +320.0

STEP = 1

def func(x, z):
    # return 30
    f0, beta = 60, 0.1
    x0, z0 = 45, 9
    return f0 * m.exp(-beta * (x - x0)**2 * (z - z0)**2)

K = 20

In [11]:
EXT_STEPS = EXT_SIZE + 1
INT_STEPS = INT_SIZE + 1

INT_OFFS = int((EXT_SIZE - INT_SIZE) / 2)

STEPS_1 = int(INT_OFFS / STEP)
STEPS_2 = int(INT_STEPS / STEP)
STEPS_3 = STEPS_1
STEPS = STEPS_1 + STEPS_2 + STEPS_3

In [12]:
# Matrix and accessors

matrix = np.zeros((STEPS, STEPS), dtype=float)
h, w = matrix.shape


# eb0, eb1 = 0, w - 1
ib0, ib1 = INT_OFFS, INT_OFFS + INT_SIZE


def is_ext_bound(y, x):
    return x in [0, w - 1] or y in [0, h - 1]

# TODO: nonsquare

def _is_hole_or_bound(y, x):
    return x >= ib0 and x <= ib1 and y >= ib0 and y <= ib1

def _is_int_bound_1dim(y, x):
    return x in [ib0, ib1] or y in [ib0, ib1]

def is_int_bound(y, x):
    return _is_hole_or_bound(y, x) and _is_int_bound_1dim(y, x)

def is_hole(y, x):
    return _is_hole_or_bound(y, x) and not _is_int_bound_1dim(y, x)

def is_target(y, x):
    return not (_is_hole_or_bound(y, x) or is_ext_bound(y, x))


def get_range1():
    return range(1, ib0)

def get_range2():
    return range(ib0, ib1 + 1)

def get_range3():
    # TODO: w/h
    return range(ib1 + 1, w - 1)

In [13]:
# Matrix printing

def print_matrix(mat):
    for j in range(h):
        for i in range(w):
            cell = '{:<8.2f}'.format(mat[j, i])
            if is_ext_bound(i, j) or is_int_bound(i, j):
                cell = colored(cell, 'green')
            elif is_hole(i, j):
                # cell = colored(cell, 'red')
                cell = ' ' * 8
            print(cell, end='')
        print()

In [14]:
# Borders setup

for j in range(h):
    matrix[j, 0] = EXT_BORDER
    matrix[j, w - 1] = EXT_BORDER
for i in range(w):
    matrix[0, i] = EXT_BORDER
    matrix[h - 1, i] = EXT_BORDER

for j in get_range2():
    matrix[j, ib0] = INT_BORDER
    matrix[j, ib1] = INT_BORDER
for i in get_range2():
    matrix[ib0, i] = INT_BORDER
    matrix[ib1, i] = INT_BORDER

In [15]:
# Creating first iteration by linear interpolation of bounds

def lerp(x, y, t):
    return (1 - t) * x + t * y

# Short intervals

for j in get_range2():
    for i in get_range1():
        matrix[i, j] = lerp(matrix[0, j], matrix[ib0, j], i / STEPS_1)
    for i in get_range3():
        matrix[i, j] = lerp(matrix[ib1, j], matrix[w - 1, j], (i - ib1) / STEPS_3)

for i in get_range2():
    for j in get_range1():
        matrix[i, j] = lerp(matrix[i, 0], matrix[i, ib0], j / STEPS_1)
    for j in get_range3():
        matrix[i, j] = lerp(matrix[i, ib1], matrix[i, h - 1], (j - ib1) / STEPS_3)

# Corners

for j in get_range1():
    for i in get_range1():
        matrix[i, j] = lerp(matrix[0, j], matrix[ib0, j], i / STEPS_1)
for j in get_range1():
    for i in get_range3():
        matrix[i, j] = lerp(matrix[ib1, j], matrix[w - 1, j], (i - ib1) / STEPS_3)
for j in get_range3():
    for i in get_range1():
        matrix[i, j] = lerp(matrix[0, j], matrix[ib0, j], i / STEPS_1)
for j in get_range3():
    for i in get_range3():
        matrix[i, j] = lerp(matrix[ib1, j], matrix[w - 1, j], (i - ib1) / STEPS_3)

$
\dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial z^2} = F(x, z) = - \dfrac{f(x, z)}{k}
$

$
\dfrac{1}{h_x^2}(u_{i+1,j} - 2u_{i,j} + u_{i-1,j}) + \dfrac{1}{h_z^2}(u_{i,j+1} - 2u_{i,j} + u_{i-1,j-1}) = F(x, z)
$

Примем $h_x = h_z = h$:

$
u_{i+1,j} - 2u_{i,j} + u_{i-1,j} + u_{i,j+1} - 2u_{i,j} + u_{i-1,j-1} = h^2 F(x, z)
$

Метод Зейделя. *Стоит рассмотреть также метод Якоби, он должен дать большую точность*.

$
u_{i,j} = \frac{1}{4}(u_{i+1,j} + u_{i-1,j} + u_{i,j+1} + u_{i-1,j-1} - h^2 F(x_i, z_j))
$

Условие окончания счёта: $\|u^{(k+1)} - u^{(k)}\| < \varepsilon$, где $\|u\| = \max_{i,j}{(u_{i,j})}$

In [16]:
# Zeidel

MAX_ITER = 3000
EPS = 1e-2

def FUNC2(x, z): return -func(x, z) / K

# Returns true if solution is found
def zeidel_iter():
    iter_max = -m.inf
    step2 = STEP * STEP
    for j in range(1, h - 1):
        for i in range(1, w - 1):
            if not is_target(i, j):
                continue
            val = 0.25 * (
                matrix[j, i + 1] +
                matrix[j, i - 1] +
                matrix[j + 1, i] +
                matrix[j - 1, i] -
                step2 * FUNC2(j * STEP, i * STEP)
            )
            diff = abs(val - matrix[j, i])
            iter_max = max(iter_max, diff)
            matrix[j, i] = val
    return iter_max < EPS

fin_iters = False
for iter in range(MAX_ITER):
    is_end = zeidel_iter()
    if is_end:
        print(f'Iterations: {iter}')
        fin_iters = True
        break

if not fin_iters:
    print(f'Iterations: {MAX_ITER} (limit exceed)')

# print_matrix(matrix)

Iterations: 287


In [17]:
# Heatmap
%matplotlib widget

XS, YS, FS = [], [], []
for j in range(h):
    for i in range(w):
        if not is_hole(i, j):
            # if i % 2 == 0 and j % 2 == 0:
            #     continue
            XS.append(i * STEP)
            YS.append(j * STEP)
            FS.append(matrix[i][j])

def show_scatter():
    plt.scatter(XS, YS, c=FS)

def show_3d():
    fig, ax = plt.figure(), plt.axes(projection='3d')
    ax.plot_trisurf(XS, YS, FS, cmap=cm.plasma)

def show_heatmap():
    vmin = min(min(x for x in row if x > 1e-3) for row in matrix)
    plt.imshow(matrix, cmap='plasma', vmin=vmin)
    hole_rect = Rectangle((INT_OFFS, INT_OFFS), INT_SIZE, INT_SIZE,
                          edgecolor='black',
                          facecolor='black')
    plt.gca().add_patch(hole_rect)
    plt.colorbar()
    plt.show()

o = widgets.Output()
with o:
    # show_scatter()
    # show_3d()
    show_heatmap()
o

Output()